In [ ]:
!pip install xgboost
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import keras 
from keras.models import Sequential 
from keras.layers import Dense

In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
df = pd.read_csv('/kaggle/input/breast-cancer-wisconsin-data/data.csv')
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
#count the number of empty values in each column
df.isna().sum()

In [ ]:
#drop NA column
df.dropna(axis=1)

In [ ]:
#Get the count of number of Malignant(M) or Beglin(B) cells
df['diagnosis'].value_counts()

In [ ]:
#Visualize
sns.countplot(df['diagnosis'], label = 'Count')

In [ ]:
df.dtypes

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder_Y = LabelEncoder()
df.iloc[:,1] = labelencoder_Y.fit_transform(df.iloc[:,1].values)

In [ ]:
df.iloc[:,1]

In [ ]:
#Create a pair plot
sns.pairplot(df.iloc[:,1:8], hue='diagnosis')

In [ ]:
df.head()

In [ ]:
#Correlation of the columns
df.corr()

In [ ]:
plt.figure(figsize = (40,40))
sns.heatmap(df.corr(), annot=True , fmt = '.0%')

In [ ]:
#Split data into independent (X) and dependent(Y) columns

X = df.iloc[:,2:31].values
Y = df.iloc[:,1].values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.25, random_state = 0)

In [ ]:
#Scale the data (Feature Scaling)
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
sc.fit_transform(X_train)
sc.fit_transform(X_test)

In [ ]:
M = {0:'Logistic Regression', 1:'Decision Tree', 2:'RandomForest', 3:'KNN', 4:'SVM', 5:'Naive Bayes', 6:'XGBoost'}
#Create a function for the models
def models(X_train,Y_train):

  # Logistic  Regression
  from sklearn.linear_model import LogisticRegression
  log = LogisticRegression(random_state = 0)
  log.fit(X_train,Y_train)

  # Decision Tree
  from sklearn.tree import DecisionTreeClassifier
  tree = DecisionTreeClassifier(random_state = 0)
  tree.fit(X_train,Y_train)

  # RandomFoest
  from sklearn.ensemble import RandomForestClassifier
  forest = RandomForestClassifier(n_estimators = 10 , criterion = 'entropy', random_state = 0)
  forest.fit(X_train,Y_train)
  
  # KNN
  from sklearn.neighbors import KNeighborsClassifier
  knn = KNeighborsClassifier(n_neighbors=5) 
  knn.fit(X_train, Y_train)  

  # "Support Vector Classifier" 
  from sklearn.svm import SVC
  clf = SVC(kernel='linear')
  clf.fit(X_train, Y_train) 

  # Naive Bayes
  from sklearn.naive_bayes import GaussianNB 
  gnb = GaussianNB() 
  gnb.fit(X_train, Y_train) 
  
  #XGBoost
  import xgboost as xgb 
  xg = xgb.XGBClassifier() 
  xg.fit(X_train, Y_train) 
  
  #Print Model Accuracy
  print('[0] LogisticRegression Training Accuracy', log.score(X_train,Y_train))
  print('[1] DecisionTree Training Accuracy', tree.score(X_train,Y_train))
  print('[2] RandomForest Training Accuracy', forest.score(X_train,Y_train))
  print('[3] KNN Training Accuracy',knn.score(X_train,Y_train))
  print('[4] SVM Training Accuracy',clf.score(X_train,Y_train))
  print('[5] Naive Bayes Training Accuracy',gnb.score(X_train,Y_train))
  print('[6] XGBoost Training Accuracy',xg.score(X_train,Y_train))
  return log,tree,forest,knn,clf,gnb,xg

In [ ]:
#Getting all the models
model = models(X_train,Y_train)

In [ ]:
#Test model Accuracy from Confusion Matrix
from sklearn.metrics import confusion_matrix
for i in (M.keys()):
  print("\n",M[i])
  cm = confusion_matrix(Y_test, model[i].predict(X_test))
  TP = cm[0][0]
  TN = cm[0][1]
  FP = cm[1][0]
  FN = cm[1][1]

  print("Training Accuracy", ((TP + FN)/(TP+FN+TN+FP))*100,"%")
  print(cm)

In [ ]:
#show another way to get metrices of the model
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score


for i in (M.keys()):
  print("\n",M[i])
  print(classification_report(Y_test, model[i].predict(X_test)))
  print(accuracy_score(Y_test, model[i].predict(X_test)))

In [ ]:
#Artificial Neural Network(ANN) Model

classifier = Sequential() 
classifier.add(Dense(activation = "relu", input_dim = 29,  
                     units = 8, kernel_initializer = "uniform")) 
classifier.add(Dense(activation = "relu", units = 14,  
                     kernel_initializer = "uniform")) 
classifier.add(Dense(activation = "sigmoid", units = 1,  
                     kernel_initializer = "uniform")) 
classifier.compile(optimizer = 'adam' , loss = 'binary_crossentropy',  
                   metrics = ['accuracy'] ) 



classifier.fit(X_train , Y_train , batch_size = 8 ,epochs = 400 ) 



Y_pred = classifier.predict(X_test) 
Y_pred = (Y_pred > 0.5) 


cmann = confusion_matrix(Y_test,Y_pred) 
print(cmann)


accuracy = (cmann[0][0]+cmann[1][1])/(cmann[0][1] + cmann[1][0] +cmann[0][0] +cmann[1][1]) 
print(accuracy*100) 


In [ ]:
accuracy = (cmann[0][0]+cmann[1][1])/(cmann[0][1] + cmann[1][0] +cmann[0][0] +cmann[1][1])  
print("Accuracy of ANN is", accuracy*100, "%") 